<a href="https://colab.research.google.com/github/hailusong/colab-god-idclass/blob/master/god_idclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Train Google Object Detection to Detect ID BBox

In [0]:
DEFAULT_HOME='/content'

## Session and Environment Verification

Establish security session with Google Cloud

In [0]:
from google.colab import auth
auth.authenticate_user()

YOUR_GCS_BUCKET='id-norm'
YOUR_PROJECT='orbital-purpose-130316'

################# RE-RUN ABOVE CELLS IF NEED TO RESTART RUNTIME #################

Verify Versions: TF, Python, IPython and prompt_toolkit (these two need to have compatible version), and protoc

In [1]:
import tensorflow as tf
print(tf.__version__)

1.13.1


In [0]:
!python -V
!ipython --version
!pip show prompt_toolkit
!protoc --version

Python 3.6.7
5.5.0
Name: prompt-toolkit
Version: 1.0.15
Summary: Library for building powerful interactive command lines in Python
Home-page: https://github.com/jonathanslenders/python-prompt-toolkit
Author: Jonathan Slenders
Author-email: UNKNOWN
License: UNKNOWN
Location: /usr/local/lib/python3.6/dist-packages
Requires: wcwidth, six
Required-by: jupyter-console, ipython
libprotoc 3.0.0


## Install Google Object Detection API in Colab
Reference is https://colab.research.google.com/drive/1kHEQK2uk35xXZ_bzMUgLkoysJIWwznYr


### Downgrade prompt-toolkit to 1.0.15
Run this **ONLY** if the Installation not Working

In [0]:
!pip install 'prompt-toolkit==1.0.15'

### Google Object Detection API Installation

In [7]:
!apt-get install -y -qq protobuf-compiler python-pil python-lxml
![ ! -e {DEFAULT_HOME}/models ] && git clone --depth=1 --quiet https://github.com/tensorflow/models.git {DEFAULT_HOME}/models
!ls {DEFAULT_HOME}/models

AUTHORS     CONTRIBUTING.md    LICENSE	 README.md  samples    WORKSPACE
CODEOWNERS  ISSUE_TEMPLATE.md  official  research   tutorials


In [8]:
import os
os.chdir(f'{DEFAULT_HOME}/models/research')
!pwd

/content/models/research


*From Wikipedia ...*: 

**protocol buffers** are a language-neutral, platform-neutral extensible mechanism for serializing structured data – think XML, but smaller, faster, and simpler. 

You define how you want your data to be structured once, then you can **use special generated source code to easily write and read your structured data to and from a variety of data streams and using a variety of languages**.

Remember **.proto defines structured data** and **protoc generates the source code** the serailize/de-serialize.

In [9]:
!protoc object_detection/protos/*.proto --python_out=.
# !ls object_detection/protos/*.proto
# !cat object_detection/protos/anchor_generator.proto
!ls {DEFAULT_HOME}/models/research/object_detection/builders/anchor*

/content/models/research/object_detection/builders/anchor_generator_builder.py
/content/models/research/object_detection/builders/anchor_generator_builder_test.py


In [0]:
import sys
sys.path.append(f'{DEFAULT_HOME}/models/research')
sys.path.append(f'{DEFAULT_HOME}/models/research/slim')

Note that ! calls out to a shell (in a **NEW** process), while % affects the **SAME** process associated with the notebook.

Since we append pathes to sys.path, we **HAVE TO** use % command to run the Python

Also it is **IMPORTANT** to have **%matplotlib inline** otherwise %run model_builder_test.py will **cause function attribute error** when accessing matplotlib.pyplot attributes from **iPython's run_line_magic** 

In [0]:
# !find . -name 'inception*' -print
%matplotlib inline

In [14]:
# If see the error 'function' object has no attribute 'called', just run the %matplotlib cell and this cell AGAIN 
%run object_detection/builders/model_builder_test.py

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.131s

OK (skipped=1)


### Pre-trained Data Prepatation
e.g. pre-trained model weights

Download, unzip and move COCO-pretrained weights data to GCS

In [15]:
import os
os.chdir(f'{DEFAULT_HOME}')
!wget http://storage.googleapis.com/download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz

--2019-03-19 14:19:45--  http://storage.googleapis.com/download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 595490113 (568M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet101_coco_11_06_2017.tar.gz’

faster_rcnn_resnet1 100%[===================>] 567.90M   156MB/s    in 3.6s    

2019-03-19 14:19:49 (156 MB/s) - ‘faster_rcnn_resnet101_coco_11_06_2017.tar.gz’ saved [595490113/595490113]



In [16]:
![ ! -e faster_rcnn_resnet101_coco_11_06_2017 ] && tar -xvf faster_rcnn_resnet101_coco_11_06_2017.tar.gz
!gsutil cp faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.* gs://{YOUR_GCS_BUCKET}/data/

faster_rcnn_resnet101_coco_11_06_2017/
faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.index
faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.meta
faster_rcnn_resnet101_coco_11_06_2017/frozen_inference_graph.pb
faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.data-00000-of-00001
faster_rcnn_resnet101_coco_11_06_2017/graph.pbtxt
Copying file://faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because

## Configuring the Object Detection Pipeline

In [34]:
![ -e {DEFAULT_HOME}/colab-god-idclass ] && git -C {DEFAULT_HOME}/colab-god-idclass pull
![ ! -e {DEFAULT_HOME}/colab-god-idclass ] && git clone --depth=1 https://github.com/hailusong/colab-god-idclass.git {DEFAULT_HOME}/colab-god-idclass
!ls -al {DEFAULT_HOME}/colab-god-idclass/configs/faster_rcnn_resnet101_pets.config

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 8 (delta 4), reused 8 (delta 4), pack-reused 0
Unpacking objects: 100% (8/8), done.
From https://github.com/hailusong/colab-god-idclass
   b2d468a..6b8440e  master     -> origin/master
Updating b2d468a..6b8440e
Fast-forward
 src/generate_tfrecord.py | 4 ++++
 1 file changed, 4 insertions(+)
-rw-r--r-- 1 root root 3735 Mar 19 14:20 /content/colab-god-idclass/configs/faster_rcnn_resnet101_pets.config


In [18]:
!gsutil cp {DEFAULT_HOME}/colab-god-idclass/configs/faster_rcnn_resnet101_pets.config gs://{YOUR_GCS_BUCKET}/data/faster_rcnn_resnet101_pets.config

Copying file:///content/colab-god-idclass/configs/faster_rcnn_resnet101_pets.config [Content-Type=application/octet-stream]...
/ [1 files][  3.7 KiB/  3.7 KiB]                                                
Operation completed over 1 objects/3.7 KiB.                                      


### Checking Your Google Cloud Storage Bucket

In [19]:
!gsutil ls gs://{YOUR_GCS_BUCKET}/data
!gsutil ls gs://{YOUR_GCS_BUCKET}/generated

gs://id-norm/data/faster_rcnn_resnet101_pets.config
gs://id-norm/data/model.ckpt.data-00000-of-00001
gs://id-norm/data/model.ckpt.index
gs://id-norm/data/model.ckpt.meta
gs://id-norm/generated/bbox-train-non-id1.csv
gs://id-norm/generated/bbox-train-non-id2.csv
gs://id-norm/generated/bbox-train-non-id3.csv
gs://id-norm/generated/bbox-train-on-dl.csv
gs://id-norm/generated/bbox-train-on-hc.csv
gs://id-norm/generated/bbox-valid-non-id1.csv
gs://id-norm/generated/bbox-valid-non-id2.csv
gs://id-norm/generated/bbox-valid-non-id3.csv
gs://id-norm/generated/bbox-valid-on-dl.csv
gs://id-norm/generated/bbox-valid-on-hc.csv
gs://id-norm/generated/pnts-train-non-id1.csv
gs://id-norm/generated/pnts-train-non-id2.csv
gs://id-norm/generated/pnts-train-non-id3.csv
gs://id-norm/generated/pnts-train-on-dl.csv
gs://id-norm/generated/pnts-train-on-hc.csv
gs://id-norm/generated/pnts-valid-non-id1.csv
gs://id-norm/generated/pnts-valid-non-id2.csv
gs://id-norm/generated/pnts-valid-non-id3.csv
gs://id-norm/g

## Prepare Our Own Data: Download, Convert and Upload

Use Google Cloud SDK gsutil to download the data file

In [20]:
# Download the file.
!gsutil cp gs://{YOUR_GCS_BUCKET}/generated.tar.gz /tmp/generated.tar.gz
!ls /tmp/*gz

Copying gs://id-norm/generated.tar.gz...
| [1 files][129.7 MiB/129.7 MiB]                                                
Operation completed over 1 objects/129.7 MiB.                                    
/tmp/generated.tar.gz


Prepare the data file (unzip, untar)

In [22]:
![[ ! -f /tmp/generated.tar && -f /tmp/generated.tar.gz ]] && gunzip /tmp/generated.tar.gz
![[ ! -e ./generated && -f /tmp/generated.tar ]] && tar xf /tmp/generated.tar
!ls generated

bbox-train-non-id1.csv	bbox-valid-on-dl.csv	pnts-valid-non-id2.csv
bbox-train-non-id2.csv	bbox-valid-on-hc.csv	pnts-valid-non-id3.csv
bbox-train-non-id3.csv	pnts-train-non-id1.csv	pnts-valid-on-dl.csv
bbox-train-on-dl.csv	pnts-train-non-id2.csv	pnts-valid-on-hc.csv
bbox-train-on-hc.csv	pnts-train-non-id3.csv	Train
bbox-valid-non-id1.csv	pnts-train-on-dl.csv	Valid
bbox-valid-non-id2.csv	pnts-train-on-hc.csv
bbox-valid-non-id3.csv	pnts-valid-non-id1.csv


In [23]:
!head -1 {DEFAULT_HOME}/generated/bbox-train-on-dl.csv | sed 's/^,/filename,/' > {DEFAULT_HOME}/train-merged.csv
!head -1 {DEFAULT_HOME}/generated/bbox-valid-on-dl.csv | sed 's/^,/filename,/' > {DEFAULT_HOME}/valid-merged.csv
!tail -q --lines=+2 {DEFAULT_HOME}/generated/bbox-train-*.csv | sed 's/\\/\//g' >> {DEFAULT_HOME}/train-merged.csv
!tail -q --lines=+2 {DEFAULT_HOME}/generated/bbox-valid-*.csv | sed 's/\\/\//g' >> {DEFAULT_HOME}/valid-merged.csv
!ls {DEFAULT_HOME}/generated
!head {DEFAULT_HOME}/train-merged.csv {DEFAULT_HOME}/valid-merged.csv

bbox-train-non-id1.csv	bbox-valid-on-dl.csv	pnts-valid-non-id2.csv
bbox-train-non-id2.csv	bbox-valid-on-hc.csv	pnts-valid-non-id3.csv
bbox-train-non-id3.csv	pnts-train-non-id1.csv	pnts-valid-on-dl.csv
bbox-train-on-dl.csv	pnts-train-non-id2.csv	pnts-valid-on-hc.csv
bbox-train-on-hc.csv	pnts-train-non-id3.csv	Train
bbox-valid-non-id1.csv	pnts-train-on-dl.csv	Valid
bbox-valid-non-id2.csv	pnts-train-on-hc.csv
bbox-valid-non-id3.csv	pnts-valid-non-id1.csv
==> /content/train-merged.csv <==
filename,bbox1_x1,bbox1_y1,bbox1_x2,bbox1_y2,label
generated/Train/non-id1/0.png,89,71,288,309,UNKNOWN
generated/Train/non-id1/1.png,147,127,427,455,UNKNOWN
generated/Train/non-id1/2.png,91,50,293,266,UNKNOWN
generated/Train/non-id1/3.png,60,133,235,439,UNKNOWN
generated/Train/non-id1/4.png,33,55,134,212,UNKNOWN
generated/Train/non-id1/5.png,45,115,160,334,UNKNOWN
generated/Train/non-id1/6.png,72,26,196,130,UNKNOWN
generated/Train/non-id1/7.png,55,83,195,295,UNKNOWN
generated/Train/non-id1/8.png,34,76,192

Upload unzip data file to GCS bucket in parallel mode (-m)

In [24]:
!gsutil cp {DEFAULT_HOME}/train-merged.csv {DEFAULT_HOME}/valid-merged.csv gs://{YOUR_GCS_BUCKET}

Copying file:///content/train-merged.csv [Content-Type=text/csv]...
Copying file:///content/valid-merged.csv [Content-Type=text/csv]...
/ [2 files][ 62.5 KiB/ 62.5 KiB]                                                
Operation completed over 2 objects/62.5 KiB.                                     


### Convert Our Label CSV Data to TF REcord
Source code is based on https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py

In [25]:
%pdb

Automatic pdb calling has been turned ON


In [35]:
import os
os.chdir(f'{DEFAULT_HOME}')

!head {DEFAULT_HOME}/train-merged.csv
!mkdir -p {DEFAULT_HOME}/coversion
!git -C {DEFAULT_HOME}/colab-god-idclass pull

# Train records first
%run {DEFAULT_HOME}/colab-god-idclass/src/generate_tfrecord.py --csv_input={DEFAULT_HOME}/train-merged.csv --output_path={DEFAULT_HOME}/coversion/train.record

filename,bbox1_x1,bbox1_y1,bbox1_x2,bbox1_y2,label
generated/Train/non-id1/0.png,89,71,288,309,UNKNOWN
generated/Train/non-id1/1.png,147,127,427,455,UNKNOWN
generated/Train/non-id1/2.png,91,50,293,266,UNKNOWN
generated/Train/non-id1/3.png,60,133,235,439,UNKNOWN
generated/Train/non-id1/4.png,33,55,134,212,UNKNOWN
generated/Train/non-id1/5.png,45,115,160,334,UNKNOWN
generated/Train/non-id1/6.png,72,26,196,130,UNKNOWN
generated/Train/non-id1/7.png,55,83,195,295,UNKNOWN
generated/Train/non-id1/8.png,34,76,192,337,UNKNOWN
Already up to date.
removing attribute csv_input
removing attribute output_path
removing attribute image_dir
total number of rows read is 978
Successfully created the TFRecords: /content/coversion/train.record


In [36]:
# Validation records second
!head {DEFAULT_HOME}/valid-merged.csv
%run {DEFAULT_HOME}/colab-god-idclass/src/generate_tfrecord.py --csv_input={DEFAULT_HOME}/valid-merged.csv --output_path={DEFAULT_HOME}/coversion/test.record

filename,bbox1_x1,bbox1_y1,bbox1_x2,bbox1_y2,label
generated/Valid/non-id1/0.png,120,117,351,400,UNKNOWN
generated/Valid/non-id1/1.png,115,159,381,545,UNKNOWN
generated/Valid/non-id1/2.png,76,205,299,630,UNKNOWN
generated/Valid/non-id1/3.png,57,74,177,243,UNKNOWN
generated/Valid/non-id1/4.png,83,98,293,387,UNKNOWN
generated/Valid/non-id1/5.png,54,49,183,211,UNKNOWN
generated/Valid/non-id1/6.png,82,41,239,195,UNKNOWN
generated/Valid/non-id1/7.png,195,117,523,434,UNKNOWN
generated/Valid/non-id1/8.png,73,98,255,359,UNKNOWN
removing attribute csv_input
removing attribute output_path
removing attribute image_dir
total number of rows read is 244
Successfully created the TFRecords: /content/coversion/test.record


In [39]:
!gsutil cp {DEFAULT_HOME}/coversion/train.record {DEFAULT_HOME}/coversion/test.record gs://{YOUR_GCS_BUCKET}/

Copying file:///content/coversion/train.record [Content-Type=application/octet-stream]...
Copying file:///content/coversion/test.record [Content-Type=application/octet-stream]...
/
Operation completed over 2 objects/130.4 MiB.                                    


## Start the Training and Evaluation Jobs on Google Cloud ML Engine

### Package the Tensorflow Object Detection code

In [0]:
import os
os.chdir(f'{DEFAULT_HOME}/models/research')

# From tensorflow/models/research/
!bash object_detection/dataset_tools/create_pycocotools_package.sh /tmp/pycocotools
!python setup.py sdist
!(cd slim && python setup.py sdist)

Three python packages will be created:
* dist/object_detection-0.1.tar.gz
* slim/dist/slim-0.1.tar.gz
* /tmp/pycocotools/pycocotools-2.0.tar.gz

In [43]:
!ls -l dist/object_detection-0.1.tar.gz
!ls -l slim/dist/slim-0.1.tar.gz
!ls -l /tmp/pycocotools/pycocotools-2.0.tar.gz

-rw-r--r-- 1 root root 439316801 Mar 19 15:27 dist/object_detection-0.1.tar.gz
-rw-r--r-- 1 root root 973662 Mar 19 15:27 slim/dist/slim-0.1.tar.gz
-rw-r--r-- 1 root root 1377835 Mar 19 15:26 /tmp/pycocotools/pycocotools-2.0.tar.gz


### Start the Training Job on Google Cloud ML

If you experience 'Permission denied on resource project ...' issue, make sure Cloud ML engine API has been enabled for the project. Use this [link](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component) to check and enable if not yet done.

In [57]:
# From tensorflow/models/research/
!gcloud config set project {YOUR_PROJECT}
!gcloud ml-engine jobs submit training `whoami`_object_detection_ids_`date +%m_%d_%Y_%H_%M_%S` \
    --runtime-version 1.9 \
    --job-dir=gs://{YOUR_GCS_BUCKET}/model_dir \
    --packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz,/tmp/pycocotools/pycocotools-2.0.tar.gz \
    --module-name object_detection.model_main \
    --region us-central1 \
    --config object_detection/samples/cloud/cloud.yml \
    -- \
    --model_dir=gs://{YOUR_GCS_BUCKET}/model_dir \
    --pipeline_config_path=gs://{YOUR_GCS_BUCKET}/data/faster_rcnn_resnet101_pets.config

Updated property [core/project].
Job [root_object_detection_ids_03_19_2019_15_59_26] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe root_object_detection_ids_03_19_2019_15_59_26

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs root_object_detection_ids_03_19_2019_15_59_26
jobId: root_object_detection_ids_03_19_2019_15_59_26
state: QUEUED


### Monitor the Training Job
To monitor in the ML Engine dashboard, click on [this link](https://console.cloud.google.com/mlengine/jobs).<br>
Or monitor it in the local Tensorboard with log data feed from your GCS bucket...

In [0]:
# This command needs to be run once to allow your local machine to access your
# GCS bucket.
!gcloud auth application-default login

!tensorboard --logdir=gs://{YOUR_GCS_BUCKET}/model_dir


The environment variable [GOOGLE_APPLICATION_CREDENTIALS] is set to:
  [/content/adc.json]
Credentials will still be generated to the default location:
  [/content/.config/application_default_credentials.json]
To use these credentials, unset this environment variable before
running your application.

Do you want to continue (Y/n)?  y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&prompt=select_account&response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&access_type=offline


Enter verification code: 4/EgFxwRF3iyx_-uDGDaSTclWuM-eLl3KrFZwoycQInHMCq-F4BFzprRU

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests
Application Default Credential